In [1]:
import os ; os.chdir(r"D:\Git_repos\Photonic_GCN")
from constants import edges_table, nodes_groups
# !pip install torch torch-geometric
import torch , pandas as pd
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [2]:
def edges_table_to_list(edges_table):
    edges = []
    for key in edges_table.keys():
        nodes_list = edges_table[key]
        for node in nodes_list:
            if ((key, node) not in edges) and ((node, key) not in edges):
                edges.append((key, node))
    return edges

In [3]:
edge_list = edges_table_to_list(edges_table)  # 107 edges
print(len(edge_list)) 
print(edge_list)


107
[('N69', 'N70'), ('N70', 'N71'), ('N71', 'N72'), ('N72', 'N73'), ('N73', 'N74'), ('N73', 'N60'), ('N74', 'N60'), ('N74', 'N94'), ('N60', 'N45'), ('N60', 'N46'), ('N60', 'N47'), ('N60', 'N61'), ('N60', 'N75'), ('N41', 'N42'), ('N42', 'N43'), ('N43', 'N44'), ('N44', 'N45'), ('N45', 'N46'), ('N46', 'N33'), ('N87', 'N86'), ('N86', 'N85'), ('N85', 'N84'), ('N84', 'N83'), ('N83', 'N82'), ('N83', 'N68'), ('N82', 'N68'), ('N82', 'N94'), ('N68', 'N55'), ('N68', 'N54'), ('N68', 'N53'), ('N68', 'N67'), ('N68', 'N81'), ('N59', 'N58'), ('N58', 'N57'), ('N57', 'N56'), ('N56', 'N55'), ('N55', 'N54'), ('N54', 'N34'), ('N33', 'N61'), ('N33', 'N47'), ('N33', 'N48'), ('N34', 'N52'), ('N34', 'N53'), ('N34', 'N67'), ('N94', 'N61'), ('N94', 'N75'), ('N94', 'N76'), ('N95', 'N80'), ('N95', 'N81'), ('N95', 'N82'), ('N95', 'N67'), ('N67', 'N81'), ('N67', 'N53'), ('N67', 'N80'), ('N67', 'N66'), ('N67', 'N52'), ('N53', 'N52'), ('N53', 'N66'), ('N81', 'N80'), ('N81', 'N66'), ('N80', 'N79'), ('N80', 'N66'), ('N

In [4]:
node_names = list(edges_table.keys())  # 51 nodes
print(node_names)

['N69', 'N70', 'N71', 'N72', 'N73', 'N74', 'N60', 'N41', 'N42', 'N43', 'N44', 'N45', 'N46', 'N87', 'N86', 'N85', 'N84', 'N83', 'N82', 'N68', 'N59', 'N58', 'N57', 'N56', 'N55', 'N54', 'N33', 'N34', 'N94', 'N95', 'N67', 'N53', 'N81', 'N80', 'N52', 'N79', 'N51', 'N65', 'N66', 'N78', 'N64', 'N50', 'N61', 'N75', 'N47', 'N62', 'N76', 'N48', 'N63', 'N77', 'N49']


In [5]:
# Node features (3 features per node)
x = torch.randn((51, 3))  # Random initialization of features for simplicity

# Network label (one-hot encoded for 37 categories)
network_label = torch.randint(0, 37, (1,)).item()
# y = F.one_hot(torch.tensor(network_label), num_classes=37).float()
y = torch.tensor([network_label])
y

tensor([18])

In [6]:
print(x.shape)
x

torch.Size([51, 3])


tensor([[-1.5804, -0.4566, -0.3652],
        [-0.3161,  0.5381, -2.0578],
        [-0.7567, -1.2979,  1.1359],
        [-0.9482,  1.1340, -0.7911],
        [-0.9610, -0.3520, -0.9133],
        [ 0.3105, -0.7836, -0.3174],
        [-1.0085,  0.4753,  0.3974],
        [ 0.0179,  1.1427, -0.8891],
        [ 1.0272,  0.6875,  0.0770],
        [-0.5854,  0.1603,  0.5870],
        [ 0.1541, -0.3508,  0.9868],
        [-0.6416,  0.5268, -0.3051],
        [-1.2935,  0.5596, -1.4141],
        [ 1.5826, -0.5121,  1.4263],
        [-0.5377, -0.5720,  0.1958],
        [ 0.0113, -1.4670, -0.0227],
        [ 0.3234, -0.7941,  0.9367],
        [ 1.9053,  0.8447, -0.1444],
        [ 0.6550, -0.1499,  1.2379],
        [ 0.4321, -0.9464, -0.3223],
        [-0.2877, -0.4066, -2.5586],
        [-0.8919,  0.2155, -1.0285],
        [-0.1306, -1.2175,  3.0869],
        [ 0.4385, -0.3481,  0.0102],
        [-0.2768, -0.4671,  1.0505],
        [-0.5800,  1.0773, -0.0244],
        [-1.4004,  1.1923, -0.1207],
 

In [7]:
y

tensor([18])

In [8]:
# Convert edge list to indices
edge_index = torch.tensor([[node_names.index(u), node_names.index(v)] for u, v in edge_list], dtype=torch.long).t().contiguous()

# Create data object
data = Data(x=x, edge_index=edge_index, y=y)

In [9]:
edge_index

tensor([[ 0,  1,  2,  3,  4,  4,  5,  5,  6,  6,  6,  6,  6,  7,  8,  9, 10, 11,
         12, 13, 14, 15, 16, 17, 17, 18, 18, 19, 19, 19, 19, 19, 20, 21, 22, 23,
         24, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30,
         30, 30, 31, 31, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 35, 36, 36,
         36, 36, 37, 37, 37, 37, 39, 39, 39, 40, 40, 40, 40, 41, 41, 42, 42, 42,
         42, 42, 43, 43, 44, 44, 45, 45, 45, 45, 45, 46, 46, 47, 47, 48, 48],
        [ 1,  2,  3,  4,  5,  6,  6, 28, 11, 12, 44, 42, 43,  8,  9, 10, 11, 12,
         26, 14, 15, 16, 17, 18, 19, 19, 28, 24, 25, 31, 30, 32, 21, 22, 23, 24,
         25, 27, 42, 44, 47, 34, 31, 30, 42, 43, 46, 33, 32, 18, 30, 32, 31, 33,
         38, 34, 34, 38, 33, 38, 35, 38, 37, 36, 37, 38, 39, 40, 37, 38, 41, 40,
         37, 38, 40, 39, 38, 41, 49, 48, 40, 48, 49, 50, 41, 50, 48, 43, 46, 45,
         47, 44, 46, 45, 45, 47, 46, 49, 48, 50, 47, 49, 48, 48, 50, 49, 50]])

In [10]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)  # Output 1 feature per node

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes):
        super(Net, self).__init__()
        self.gcn = GCN(in_channels, hidden_channels)
        self.fc1 = torch.nn.Linear(51, 128)  # 51 nodes, each with 1 feature from GCN
        self.fc2 = torch.nn.Linear(128, num_classes)

    def forward(self, data):
        x = self.gcn(data)  # GCN output, shape [num_nodes, 1]
        x = x.view(1, -1)  # Flatten the node features, shape [1, num_nodes]
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  # Output probabilities

# Model parameters
in_channels = 3
hidden_channels = 16
num_classes = 37

# Initialize model
model = Net(in_channels, hidden_channels, num_classes)


In [11]:
# Loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
def train(data, model, optimizer, criterion, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Training the model
train(data, model, optimizer, criterion)


Epoch 0, Loss: 3.800077199935913
Epoch 10, Loss: 0.0
Epoch 20, Loss: 0.0
Epoch 30, Loss: 0.0
Epoch 40, Loss: 0.0
Epoch 50, Loss: 0.0
Epoch 60, Loss: 0.0
Epoch 70, Loss: 0.0
Epoch 80, Loss: 0.0
Epoch 90, Loss: 0.0


In [12]:
data

Data(x=[51, 3], edge_index=[2, 107], y=[1])

Data preparation

In [13]:
BATCH_SIZE = 8

In [39]:
df = pd.read_excel('data.xlsx')

X = df.iloc[:, :18].values
y = df.iloc[:, 18:19].values

scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32).to(device)

train_size = int(0.7 * len(X_tensor))
val_size = int(0.15 * len(X_tensor))
test_size = len(X_tensor) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(TensorDataset(X_tensor, y_tensor), [train_size, val_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [42]:
X.shape

(8015, 18)

In [15]:
df

,dx1,dx2,dx3,dx4,dx5,dx6,dy1,dy2,dy3,dy4,...,dy6,dr1,dr2,dr3,dr4,dr5,dr6,Q,t_q_max,lambda_q_max
0,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,918.130324,0.554622,0.000001
1,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.750000e-08,2.500000e-08,1846.114296,0.682024,0.000001
2,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,3.000000e-08,2.500000e-08,660.817503,1.089219,0.000002
3,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.750000e-08,2.500000e-08,1846.114296,0.682024,0.000001
4,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,3.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.750000e-08,2.500000e-08,1846.114296,0.682024,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8010,-4.000000e-08,3.500000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,1204.550299,0.521705,0.000001
8011,-4.000000e-08,4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,1306.557266,0.549646,0.000001
8012,-4.000000e-08,4.500000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,618.706366,0.701339,0.000002
8013,-4.000000e-08,5.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,610.208722,0.826126,0.000002


In [43]:
df["Q"][0]

918.1303236596783

In [19]:
nodes2group = dict()

for g in nodes_groups:
    for node in nodes_groups[g]:
        nodes2group[node] = g


In [44]:
xs_list = []
ys_list = []

for n in range(len(df)):
    x = np.zeros((51, 3))
    y = np.array(df["Q"][n])
    for idx, node in enumerate(node_names):
        group = nodes2group[node][1:]
        node_features = np.array((df["dx" + group][n], df["dy" + group][n], df["dr" + group][n],))
        x[idx] = node_features
    xs_list.append(x)
    ys_list.append(y)

data_x = np.stack(xs_list, axis=0)
data_y = np.stack(ys_list, axis=0)
print(data_x.shape)
print(data_y.shape)

(8015, 51, 3)
(8015,)


In [45]:
data_y 

array([ 918.13032366, 1846.11429554,  660.81750282, ...,  618.70636616,
        610.20872204,  597.35792412])